# Linear regression walkthrough
There are many ways to calculate regression, however using matrices is one of my favorite ways because of how scalable it is to higher orders as well as how quick it is when the A matrix is invertible upon projection to the nearest subspace.  
  
$$Ax = b$$
Let matrix A be a matrix with height of the number of data points. A has a column of ones, a column of the x values, and an extra column for every higher order desired in our line of best fit where the column will be the x values to that order. For instance, for a line of best fit with order to the power of 2, the A matrix will have a column of ones, of x values, and of each of the x values squared. For a line of best fit with order of 3, just add another column of each of the x values to the power of 3. Let x be of height of the number of columns in A. x is unknown. We are trying to solve for x. The solution for x will be called x with a hat on top, or x hat. x hat will be the coefficients of each x in our line of best fit. Finally, be is simply a matrix with one column that contains all of the y coordinates. 
$$Ax = b$$
Solve by multiplying both sides by A transpose -----------------------------------------------------------------------------------
$$A^TA\hat{x} = A^Tb$$
Simplify -------------------------------------------------------------
$$\hat{x} = ((A^TA)^{-1})(A^Tb)$$
When ATA is not invertible, take the pseudoinverse to approximate the closest inverse. In all cases, one is safest to simply take the pseudoinverse becasuse if the matrix is invertible then the pseudoinverse will just be the inverse. It is more computationally intensive, but not much more.

Then to calculate Error for metrics like R squared or MAE etc, you can use the fact that the error matrix equals
$$e = b - p$$
Where -------------------------------------------------------------
$$p = A\hat{x}$$  
  
Next to calculate R², we use the following equation ---------------------------------------------------------
$$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$$
$$=$$
$$R^2 = 1 - \frac{SSR}{SST}$$
The numerator of this equation is called the sum squared regression (SSR). It is the sum of the squared differences between all of the actual y data points and their predicted values on the lne of best fit.  
The denominator is called the total sum of squares (SST). It is the sum of the squared differences between all of the actual y data points and the mean of all of the y data points.

In [59]:

import numpy as np
import plotly.graph_objects as go

def convert_text(text):
    text = str(text)
    normal_chars = "0123456789"
    superscript_chars = "⁰¹²³⁴⁵⁶⁷⁸⁹"

    mapping = str.maketrans(normal_chars, superscript_chars)

    converted_text = text.translate(mapping)
    return converted_text

def create_dist(x_mean: float, x_std: float, y_mean: float, y_std: float, num_of_points: int, order: int):
    # Generate your x and y data points
    x = np.random.normal(x_mean, x_std, num_of_points)
    y = .5*(x**3) + x**2 + 100*np.random.normal(y_mean, y_std, num_of_points)

    # Add a column of ones to account for the y intercept constant
    A = np.column_stack((np.ones(x.shape[0]), x))
    for i in range(2, order + 1):
        A = np.column_stack((A, x**i))


    """START OF CALCULATIONS"""

    # Generating x hat for Ax = b
    def generate_x_hat(A: np.array):
        x_hat = np.linalg.pinv(A.T.dot(A)).dot(A.T).dot(y)
        return x_hat

    x_hat = generate_x_hat(A)

    # Calculate error 
    # y - p where p = A(x_hat)
    p = A.dot(x_hat)
    error = y - p

    # Calculate R^2 -- SSR/SST 
    # SSR = sum squared regression (SSR) = sum of errors squared
    # SST = total sum of squares (SST) = (sum of y values of points) - (mean of all y values)
    ## mean of y points
    
    ssr = np.sum(error ** 2)
    sst = np.sum((y - np.mean(y)) ** 2)
    r_squared = 1-(ssr/sst)
    #print(f"ssr: {ssr}")
    #print(f"sst: {sst}")
    #print(f"R^2 ssr/sst: {r_squared}")
    

    """END OF CALCULATIONS"""


    """CREATING PLOTS"""
    # Plotting with Plotly
    fig = go.Figure()

    # Generate x values for the curve
    x_curve = np.linspace(np.min(x), np.max(x), 100)
    y_curve = np.zeros_like(x_curve)  # Initialize y_curve with zeros

    # Add terms for each power of x_curve
    for i in range(order + 1):
        y_curve += x_hat[i] * (x_curve ** i)  
    
    # Create the string equation 
    equation = "y = "
    for i in range(order, -1, -1):
        if i == 0:
            equation += f"{x_hat[i]:.2f}"
        elif i == 1:
            equation += f"{x_hat[i]:.2f}x + "
        else:
            equation += f"{x_hat[i]:.2f}x{convert_text(i)} + "
        if i%3 == 0:
            equation += "<br>"

    # Scatter plot
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Data'))
    # Line of best fit
    fig.add_trace(go.Scatter(x=x_curve, y = y_curve, mode='lines', name=f"{equation}R² = {round(r_squared, 3)}"))

    # Update layout for better visualization
    fig.update_layout(
        title="Interactive Scatter Plot with Line of Best Fit",
        xaxis_title="X-axis",
        yaxis_title="Y-axis"
    )

    fig.show()

create_dist(1, 10, 1, 10, 500, 3)